#### Notebook responsável por extrair os dados da API da steamdb



In [5]:
!pip install pyspark
!pip install requests
!pip install beautifulsoup4
!pip install google-cloud-bigquery
!pip install google-cloud-storage

#### Importa bibliotecas responsáveis pelo ETL

In [6]:
import requests
from pyspark.sql.functions import *
from pyspark.sql import *
from pyspark.sql.types import *
import pandas as pd
from google.colab import files

In [7]:
# Inicia a sessão Spark
spark = SparkSession.builder.appName("SteamAPI").getOrCreate()

## Função que retorna as categorias dos jogos listados na Steam

In [8]:
def listaCategorias(url: str, params: dict) -> DataFrame:
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()

        categorias = data.get("response", {}).get("tags", [])


        schema = StructType([
            StructField("tagid", IntegerType(), True),
            StructField("name", StringType(), True),
        ])

        # Criação do DataFrame
        df = spark.createDataFrame(categorias, schema=schema)
        return df
    else:
        print("Erro:", response.status_code, response.text)
        return None

url = "https://api.steampowered.com/IStoreService/GetMostPopularTags/v1/"
parametros = {
    "key": "8700AD359A4579244D1C25C9C673D947"
}

df_categorias = listaCategorias(url, parametros)


### Tratamento nas colunas que estão em Inglês

In [9]:
df_categorias = df_categorias.withColumnRenamed("tagid", "idCategoria")\
                             .withColumnRenamed("name", "Categoria")

In [10]:
df_categorias.show()

+-----------+--------------+
|idCategoria|     Categoria|
+-----------+--------------+
|        492|         Indie|
|         19|        Action|
|         21|     Adventure|
|        597|        Casual|
|        122|           RPG|
|        599|    Simulation|
|       4182|  Singleplayer|
|          9|      Strategy|
|        493|  Early Access|
|        113|  Free to Play|
|       3871|            2D|
|       4191|            3D|
|       4166|   Atmospheric|
|       4305|      Colorful|
|       1742|    Story Rich|
|       1684|       Fantasy|
|       3859|   Multiplayer|
|       3834|   Exploration|
|       1664|        Puzzle|
|       3964|Pixel Graphics|
+-----------+--------------+
only showing top 20 rows



#### Agrupando por Similaridade para saber quantos jogos Multiplayer tem disponíveis

In [11]:
df_multiplayer = df_categorias.filter(df_categorias["Categoria"].like("%Multi%"))
df_multiplayer.show(truncate=False)

+-----------+------------------------+
|idCategoria|Categoria               |
+-----------+------------------------+
|3859       |Multiplayer             |
|128        |Massively Multiplayer   |
|6971       |Multiple Endings        |
|7368       |Local Multiplayer       |
|17770      |Asynchronous Multiplayer|
+-----------+------------------------+



#### Função que retorna todos os jogos disponíveis na Steam de acordo com a API do steamdb

In [12]:
def listaJogos(url: str, params: dict) -> DataFrame:
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()


        jogos = data.get("response", {}).get("apps", [])

        schema = StructType([
            StructField("appid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("last_modified", LongType(), True),
            StructField("price_change_number", IntegerType(), True)
        ])

        # Criação do DataFrame
        df_jogos = spark.createDataFrame(jogos, schema=schema)
        return df_jogos
    else:
        print("Erro:", response.status_code, response.text)
        return None

url = "https://api.steampowered.com/IStoreService/GetAppList/v1/"
params = {
    "key": "8700AD359A4579244D1C25C9C673D947"
}

df_jogos = listaJogos(url, params)


In [13]:
df_jogos.show()

+-----+--------------------+-------------+-------------------+
|appid|                name|last_modified|price_change_number|
+-----+--------------------+-------------+-------------------+
|   10|      Counter-Strike|   1729702322|           24324481|
|   20|Team Fortress Cla...|   1729702194|           24324481|
|   30|       Day of Defeat|   1729702105|           24324481|
|   40|  Deathmatch Classic|   1729702150|           24324481|
|   50|Half-Life: Opposi...|   1729702282|           24324481|
|   60|            Ricochet|   1729701834|           24324481|
|   70|           Half-Life|   1729702360|           24324481|
|   80|Counter-Strike: C...|   1729702001|           24324481|
|  130|Half-Life: Blue S...|   1721932676|           24324481|
|  220|         Half-Life 2|   1727742736|           24853235|
|  240|Counter-Strike: S...|   1729702412|           24324481|
|  300|Day of Defeat: So...|   1729702240|           24324481|
|  320|Half-Life 2: Deat...|   1729702057|           24

#### Tratamento nas colunas

In [14]:
df_jogos = df_jogos.withColumnRenamed("appid", "idJogo")\
                             .withColumnRenamed("name", "Jogo")\
                             .withColumnRenamed("last_modified", "DataUltimaModificacao")\
                             .withColumnRenamed("price_change_number", "numeroAlteracoesPreco")

if df_jogos.schema["DataUltimaModificacao"].dataType != TimestampType():
  df_jogos = df_jogos.withColumn("DataUltimaModificacao", from_unixtime("DataUltimaModificacao").cast("timestamp"))

In [15]:
df_jogos.show()

+------+--------------------+---------------------+---------------------+
|idJogo|                Jogo|DataUltimaModificacao|numeroAlteracoesPreco|
+------+--------------------+---------------------+---------------------+
|    10|      Counter-Strike|  2024-10-23 16:52:02|             24324481|
|    20|Team Fortress Cla...|  2024-10-23 16:49:54|             24324481|
|    30|       Day of Defeat|  2024-10-23 16:48:25|             24324481|
|    40|  Deathmatch Classic|  2024-10-23 16:49:10|             24324481|
|    50|Half-Life: Opposi...|  2024-10-23 16:51:22|             24324481|
|    60|            Ricochet|  2024-10-23 16:43:54|             24324481|
|    70|           Half-Life|  2024-10-23 16:52:40|             24324481|
|    80|Counter-Strike: C...|  2024-10-23 16:46:41|             24324481|
|   130|Half-Life: Blue S...|  2024-07-25 18:37:56|             24324481|
|   220|         Half-Life 2|  2024-10-01 00:32:16|             24853235|
|   240|Counter-Strike: S...|  2024-10

### Cria uma coluna que verifica, de acordo com o número de alterações de preço, qual jogo tem alta frequência de alteração

In [16]:
df_jogos = df_jogos.withColumn(
    "frequenciaAlteracaoPreco",
    when(df_jogos["numeroAlteracoesPreco"] < 10, "Baixa")
    .when((df_jogos["numeroAlteracoesPreco"] >= 10) & (df_jogos["numeroAlteracoesPreco"] < 50), "Média")
    .otherwise("Alta")
)

In [17]:
df_jogos.show()

+------+--------------------+---------------------+---------------------+------------------------+
|idJogo|                Jogo|DataUltimaModificacao|numeroAlteracoesPreco|frequenciaAlteracaoPreco|
+------+--------------------+---------------------+---------------------+------------------------+
|    10|      Counter-Strike|  2024-10-23 16:52:02|             24324481|                    Alta|
|    20|Team Fortress Cla...|  2024-10-23 16:49:54|             24324481|                    Alta|
|    30|       Day of Defeat|  2024-10-23 16:48:25|             24324481|                    Alta|
|    40|  Deathmatch Classic|  2024-10-23 16:49:10|             24324481|                    Alta|
|    50|Half-Life: Opposi...|  2024-10-23 16:51:22|             24324481|                    Alta|
|    60|            Ricochet|  2024-10-23 16:43:54|             24324481|                    Alta|
|    70|           Half-Life|  2024-10-23 16:52:40|             24324481|                    Alta|
|    80|Co

### Cria uma coluna que verifica a diferença de dias entre a última atualização e o dia atual.


*   Jogos Recentes = Modificados nos últimos 30 dias
*   Jogos Antigos = Modificados nos últimos 365 dias



In [18]:
df_jogos = df_jogos.withColumn("diasDesdeUltimaModificacao", datediff(current_date(), "DataUltimaModificacao"))

jogos_recentes = df_jogos.filter(df_jogos["diasDesdeUltimaModificacao"] < 30)
jogos_antigos = df_jogos.filter(df_jogos["diasDesdeUltimaModificacao"] > 365)


jogos_recentes.show()
jogos_antigos.show()


+------+--------------------+---------------------+---------------------+------------------------+--------------------------+
|idJogo|                Jogo|DataUltimaModificacao|numeroAlteracoesPreco|frequenciaAlteracaoPreco|diasDesdeUltimaModificacao|
+------+--------------------+---------------------+---------------------+------------------------+--------------------------+
|    10|      Counter-Strike|  2024-10-23 16:52:02|             24324481|                    Alta|                        15|
|    20|Team Fortress Cla...|  2024-10-23 16:49:54|             24324481|                    Alta|                        15|
|    30|       Day of Defeat|  2024-10-23 16:48:25|             24324481|                    Alta|                        15|
|    40|  Deathmatch Classic|  2024-10-23 16:49:10|             24324481|                    Alta|                        15|
|    50|Half-Life: Opposi...|  2024-10-23 16:51:22|             24324481|                    Alta|                    

## Função que retorna as recompensas que podem ser resgatadas dentro dos jogos

In [138]:
def recompensasDisponiveis(url):
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Erro: {response.status_code}")
        return None

    data = response.json().get("response", {}).get("definitions", [])

    schema = StructType([
        StructField("appid", IntegerType(), True),
        StructField("defid", IntegerType(), True),
        StructField("type", IntegerType(), True),
        StructField("community_item_class", IntegerType(), True),
        StructField("community_item_type", IntegerType(), True),
        StructField("point_cost", StringType(), True),
        StructField("timestamp_created", IntegerType(), True),
        StructField("timestamp_updated", IntegerType(), True),
        StructField("active", BooleanType(), True),
        StructField("item_title", StringType(), True),
        StructField("item_description", StringType(), True),
        StructField("item_image_small", StringType(), True),
        StructField("item_image_large", StringType(), True),
        StructField("animated", BooleanType(), True)
    ])

    dados_recompensas = [
        {
            "appid": item.get("appid"),
            "defid": item.get("defid"),
            "type": item.get("type"),
            "community_item_class": item.get("community_item_class"),
            "community_item_type": item.get("community_item_type"),
            "point_cost": item.get("point_cost"),
            "timestamp_created": item.get("timestamp_created"),
            "timestamp_updated": item.get("timestamp_updated"),
            "active": item.get("active"),
            "item_title": item["community_item_data"].get("item_title"),
            "item_description": item["community_item_data"].get("item_description", ""),
            "item_image_small": item["community_item_data"].get("item_image_small"),
            "item_image_large": item["community_item_data"].get("item_image_large"),
            "animated": item["community_item_data"].get("animated")
        }
        for item in data
    ]

    # Criação do DataFrame
    df_recompensas = spark.createDataFrame(dados_recompensas, schema=schema)
    return df_recompensas

url = "https://api.steampowered.com/ILoyaltyRewardsService/QueryRewardItems/v1/"
df_recompensas = recompensasDisponiveis(url)


#### Tratamento e DROP de colunas não utilizadas

In [139]:
df_recompensas = df_recompensas.withColumnRenamed("appid", "idJogo")\
                               .withColumnRenamed("defid", "idRecompensa")\
                               .withColumnRenamed("type", "Tipo")\
                               .withColumnRenamed("community_item_class", "classeItemComunitario")\
                               .withColumnRenamed("community_item_type", "tipoItemComunitario")\
                               .withColumnRenamed("point_cost", "custoPontos")\
                               .withColumnRenamed("timestamp_created", "dataCriacao")\
                               .withColumnRenamed("timestamp_updated", "dataAtualizacao")\
                               .withColumnRenamed("active", "Ativo")\
                               .withColumnRenamed("item_title", "tituloItem")\
                               .withColumnRenamed("item_description", "descricaoItem")\
                               .withColumnRenamed("item_image_small", "imagemItemPequena")\
                               .withColumnRenamed("item_image_large", "imagemItemGrande")\
                               .withColumnRenamed("animated", "Animado")

df_recompensas = df_recompensas.drop("imagemItemPequena", "imagemItemGrande")

In [140]:
# Tratando os id's que não estão no padrão (Contém emoji no título)
df_recompensas = df_recompensas.filter(col("idJogo") != '2291700')

In [141]:
if df_recompensas.schema["dataCriacao"].dataType != TimestampType():
    df_recompensas = df_recompensas.withColumn("dataCriacao", from_unixtime("dataCriacao").cast("timestamp"))

if df_recompensas.schema["dataAtualizacao"].dataType != TimestampType():
    df_recompensas = df_recompensas.withColumn("dataAtualizacao", from_unixtime("dataAtualizacao").cast("timestamp"))

In [142]:
df_recompensas.show()

+-------+------------+----+---------------------+-------------------+-----------+-------------------+-------------------+-----+-------------------+-------------+-------+
| idJogo|idRecompensa|Tipo|classeItemComunitario|tipoItemComunitario|custoPontos|        dataCriacao|    dataAtualizacao|Ativo|         tituloItem|descricaoItem|Animado|
+-------+------------+----+---------------------+-------------------+-----------+-------------------+-------------------+-----+-------------------+-------------+-------+
|1191660|      277350|   1|                   11|                 15|       1000|2024-11-05 22:28:05|2024-11-05 22:30:32| true|       Moon Running|             |   true|
|3263320|      336639|   1|                    3|                  8|        500|2024-11-05 12:31:12|2024-11-06 20:56:53| true|        The Workers|             |  false|
|2624050|      336452|   1|                    3|                  7|        500|2024-11-04 23:29:26|2024-11-06 04:55:37| true|Victoria Background|   

#### Classificando a quantidade em custos

*   Menos que 500 pontos = Barato
*   Maior ou igual a 500 pontos = Caro



In [143]:
df_recompensas = df_recompensas.withColumn("categoriaCusto",
                                           when(df_recompensas["custoPontos"] < 500, "Barato")
                                           .when((df_recompensas["custoPontos"] >= 500) & (df_recompensas["custoPontos"] <= 1500), "Moderado")
                                           .otherwise("Caro"))


#### Download de todas as tabelas para CSV e futuramente consumo no Big Query

In [42]:
# Para esse teste, decidi fazer o download direto do arquivo para a minha máquina local e posteriormente inserindo os dados para dentro do bucket para mostrar a facilidade de integração com o BigQuery.
df_categorias_pd = df_categorias.toPandas()
df_categorias_pd.to_csv('categorias.csv', index=False)

df_multiplayer_pd = df_multiplayer.toPandas()
df_multiplayer_pd.to_csv('multiplayer.csv', index=False)

df_jogos_pd = df_jogos.toPandas()
df_jogos_pd.to_csv('jogos.csv', index=False)

jogos_recentes_pd = jogos_recentes.toPandas()
jogos_recentes_pd.to_csv('jogos_recentes.csv', index=False)

jogos_antigos_pd = jogos_antigos.toPandas()
jogos_antigos_pd.to_csv('jogos_antigos.csv', index=False)

df_recompensas_pd = df_recompensas.toPandas()
df_recompensas_pd.to_csv('recompensas.csv', index=False)


files.download('categorias.csv')
files.download('multiplayer.csv')
files.download('jogos.csv')
files.download('jogos_recentes.csv')
files.download('jogos_antigos.csv')
files.download('recompensas.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>